# Initial Set Up: Data Pre-processing Step
This notebook is for the initial set up of the LLM application that will be the same for each model.

The focus is on the data-pre processing step of the RAG pipeline and getting the data into the vector database.

We need to put the data of the SMU Catalog of 2023-2024 into Qdrant which is a cloud vector database. This will allow the language model to access and retrieve the necessary information.

There are many changes that can be done at this step to alter how the text goes into the vector database (ex: different text splitters, document loaders, retrievers, etc.)

We will install some of the necessary dependancies now and the rest along the way throughout the notebook.

In [ ]:
%pip install openai pypdf qdrant-client langchain python-dotenv tiktoken langchain-openai pandas

In [1]:
# Set up to initialize API keys from .env file into the
import os
from dotenv import find_dotenv, load_dotenv

# Load environment variables from the .env files
load_dotenv(find_dotenv(filename='SURF-Project_Optimizing-PerunaBot/setup/.env'))

True

## PDF Breakdown
If you take a look at the pdfs in the data folder, just the catalog pdf alone is over 800 pages long! To upload it into the vector databse, we have to 1st get all the text from the documents.

First we will do the PDFs, then later on the CSV of the FAQs with a different method.

In [2]:
# langchain imports
from langchain.schema import Document
from langchain_community.document_loaders import PyPDFLoader

# file paths to the two PDFs we're using
pdf_paths = ['C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/20232024 Undergraduate Catalog91123.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/Official University Calendar 2023-2024.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/2023_PerunaPassport.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMU Student Handbook 23-24.pdf',
             'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMUCampusGuideFactsMap.pdf'
             ]

def load_pdfs_with_langchain(pdf_paths):
    documents = []
    for path in pdf_paths:
        try:
            # Use LangChain's PyPDFLoader to load the PDF
            loader = PyPDFLoader(path)
            # Load and pase the PDF into document instances
            pdf_doc = loader.load()
            # Insert pdf into documents list variable
            documents.extend(pdf_doc)
        except Exception as e:
            print(f"Error loading {path}: {e}")
    return documents

#Load PDF documents using the function
docs = load_pdfs_with_langchain(pdf_paths)

print(len(docs))
print(docs[0].page_content[0:100])
print(docs[0].metadata)

1015
 
 
 
 
 
 
Southern Methodist University 
General Information 
Undergraduate Catalog  
2023 -2024  
{'source': 'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/20232024 Undergraduate Catalog91123.pdf', 'page': 0}


Here we are initializing an API connection to the Qdrant vector database from https://qdrant.tech/

In [3]:
from qdrant_client import qdrant_client
from qdrant_client.http import models

In [4]:
# Initializing Qdrant host URL and API key
qdrant_host = os.environ['QDRANT_HOST']
qdrant_api_key = os.environ['QDRANT_API_KEY']

#Initialize Qdrant Client
client = qdrant_client.QdrantClient(
    url=qdrant_host, 
    api_key = qdrant_api_key,
)

Now we are going to create the 1st collection of vectors and the vectorstore inside the database. Eventually, we will have more than one collection to see how changes to how the data is uploaded and retrieved affects the accuracy and other evaluation metrics.

This is a very important step because an LLM application is only as good as its data and the documents it retrieves to create an answer.

Since we will have more than one collection within the vector database, we will just create a function that will allow us to create a new vectorstore collection when needed.

In [5]:
# function to create a vector store based on the collection name
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant

# Initializing OpenAI API key for embeddings and later use
openai_api_key = os.environ['OPENAI_API_KEY']

def create_vectorstore(qdrant_collection_name):
    
    vectors_config = models.VectorParams(
   size=1536, #for OpenAI
   distance=models.Distance.COSINE
   )
    
    client.create_collection(
   collection_name = qdrant_collection_name,
   vectors_config=vectors_config,   
    )

    vector_store = Qdrant(
        client=client, 
        collection_name=qdrant_collection_name, 
        embeddings=OpenAIEmbeddings(),
    )
  
    return vector_store

In [8]:
# create 1st collection of vectors

qdrant_collection_1 = os.environ['QDRANT_COLLECTION_1']
vector_store_1 = create_vectorstore(qdrant_collection_1)

This is where the experiment begins! Now we have all the text from the pdfs in the pdfs_doc_text variable, the text needs to be split into chunks using langchain text splitters to be turned into vectors using the OpenAI Embeddings Model.

We are going to test two methods:
1. Parent Doucment Retriever method with the **RecursiveCharacterTextSplitter**

2. Semantic Chunking method using the **Semantic Text Splitter**

Since this will generate two different types of chunks, we will but them in two different collections within the vector database.

In [9]:
# Parent Document Retriever Method
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

child_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=25, 
                                                length_function=len, add_start_index=True) 

parent_splitter = RecursiveCharacterTextSplitter(chunk_size=750, chunk_overlap=50, 
                                                length_function=len, add_start_index=True)  

# storage for parent splitter
store = InMemoryStore()

# retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=vector_store_1, 
    docstore=store, 
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
    )
# adding  documents into the Qdrant vector database in the 1st collection
parent_retriever.add_documents(docs)

# testing the retriever
parent_retriever.invoke("How many credit hours is a major in Computer Science?")


[Document(page_content='Lyle to meet AME or Leadership requirements with adviser approval.  \nComputer Science Minor \nA student majoring in computer engineering may not minor in computer science.  \nRequirements for the Minor  \nThe following computer science courses are required:  \n• CS 1341 -  Principles of Computer Science I  \n• CS 1342 -  Principles of Computer Science II  \n• CS 2341 -  Data Structures  \n• CS 2353 -  Discrete Computational Structures  \nElective Courses \nElective courses can be any six hours of CS  courses numbered 3000 or above as approved by the computer science \nminor adviser.  \nTotal: 18 Credit Hours  \nComputer Science Courses  \nCS 1340 -  Introduction to Computing Concepts  \nCredits:  3', metadata={'source': 'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/20232024 Undergraduate Catalog91123.pdf', 'page': 533, 'start_index': 1391}),
 Document(page_content='Senior Capstone (3 Credit Hours)  \n• CRCP 5301 -  Creative Co

Now for the 2nd method using the Semantic Text Splitter which splits the text based on the meaning within each sentence for more granular control of retrieval.

For this one, we will use the Ensemble Retriever which allows us to combine the results of multiple retrievers, giving them different weights. Within the Ensemble retriever we will use: 

- BM25 Retriever which a retrieval method used by search engines
- Base retriever that comes with using the vectorstore as a retriever


In [10]:
# semantic text splitting method
# do '%pip install langchain_experimental' if needed
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

semantic_text_splitter = SemanticChunker(
    OpenAIEmbeddings(), 
    breakpoint_threshold_type="percentile")

semantic_docs = semantic_text_splitter.split_documents(docs)
print(semantic_docs[0].page_content)
print(len(semantic_docs))


 
 
 
 
 
 
Southern Methodist University 
General Information 
Undergraduate Catalog  
2023 -2024   
2444


In [11]:
# creating another instance of a vector store with a new collection using the function we made earlier
qdrant_collection_2 = os.environ['QDRANT_COLLECTION_2']
vector_store_2 = create_vectorstore(qdrant_collection_2)

In [12]:
# %pip install rank_bm25
from langchain.retrievers import EnsembleRetriever, BM25Retriever
# we already imported the Qdrant vector store and OpenAI embeddings in a previous step

bm25_retriever = BM25Retriever.from_documents(semantic_docs)

# vector_store_2.from_documents(semantic_docs, OpenAIEmbeddings())
vector_store_2_retriever = vector_store_2.as_retriever()

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vector_store_2_retriever], weights=[0.7, 0.3]
)

# adding the semantically split docs into the vector store
vector_store_2_retriever.add_documents(semantic_docs)

ensemble_retriever.invoke("How many credit hours is a major in Computer Science?")

[Document(page_content="555 \n Electrical and Computer Engineering, B.S.E.C.E., with a \nSpecialization in Electrical Engineering  \nThe electrical and computer engineering curriculum is administered by the Department of Electrical and Computer \nEngineering. SMU Degree Requirements  \nAn SMU undergraduate degree requires a minimum of 120 credit hours and must include completion of the \nUniversity's Common Curriculum, one major and a combination of electives and/or other majors or minors. Completion of certain majors requires more than 120 hours to finish the degree.", metadata={'source': 'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/20232024 Undergraduate Catalog91123.pdf', 'page': 554}),
 Document(page_content="Students may pursue a second degree program in math or physics. Students wishing to obtain a second degree in mathematics or physics should contact the respective departments in Dedman College to discuss the requirements. Electrical and Comp

We are going to have a base option that splits the text using just the RecursiveCharacterTextSplitter like the [original repo](https://github.com/yawbtng/SMUChatBot_Project/blob/main/main.py) does. From there, we're creating a third vector store collection to upload this text into using the vector store as the retriver.

In [13]:
base_text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, 
                                                length_function=len, add_start_index=True)  

normal_split_docs = base_text_splitter.split_documents(docs)

# checking result
print(normal_split_docs[0].page_content)
print(len(normal_split_docs))

Southern Methodist University 
General Information 
Undergraduate Catalog  
2023 -2024
7323


In [14]:

# getting the collection name of the third vector store
qdrant_collection_0 = os.environ['QDRANT_COLLECTION_0']

# creating the third vector store and retriever
vector_store_0 = create_vectorstore(qdrant_collection_0)
vector_store_0_retriever = vector_store_0.as_retriever()

# adding the recursively split docs into the vector store
vector_store_0_retriever.add_documents(normal_split_docs)

# testing the retriever
vector_store_0_retriever.invoke("How many credit hours is a major in Computer Science?")

[Document(page_content='up to two computer science courses, not including CS 1341 or CS 1342.  \nJunior Milestone (3 Credit Hours)  \n• CRCP 4301 -  Junior Milestone  \nSenior Capstone (3 Credit Hours)  \n• CRCP 5301 -  Creative Computing Capstone  \nor \n• CS 5352 -  Senior Design II  \nTotal for the Major Only: 36 Credit Hours  \nCreative Computing Minor', metadata={'page': 697, 'source': 'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/20232024 Undergraduate Catalog91123.pdf', 'start_index': 1294, '_id': '2bd28e21-1e15-4a49-bceb-80ea1fd56536', '_collection_name': 'smu-data_0'}),
 Document(page_content="Total for the Major Only: 60 Credit Hours  \nNote:  All computer science majors must earn a grade of  C- or better in the computer science core courses and CS \n2353 in fulfillment of the requirements for the major.  \nComputer Science, B.S.  \nSMU Degree Requirements  \nAn SMU undergraduate degree requires a minimum of 120 credit hours and must include

## CSV Breakdown
Now we are going to work on uploading the excel file of 115 FAQs into each vector database collection. There are multiple sheets within the excel file so we're going to have to turn each sheet into a CSV using the pandas library and then use langchain's CSV loader to turn them into langchain documents.

The CSVs you see there now were created by iterating through the xlsx file so you can delete those if you want to see them being recreated (be careful not to delete the xlsx file)

In [15]:
# using the pandas library to work with the excel file and convert it to a data frame
import pandas as pd

# Load the Excel file
excel_path = 'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMU FAQs.xlsx'
xlsx = pd.ExcelFile(excel_path)

# checking to see if loading the file worked
print(xlsx.sheet_names)

# Iterate through each sheet and save as a CSV file
csv_files = []
for sheet_name in xlsx.sheet_names:
    # Read the entire sheet to extract the metadata from cell A1
    sheet_df = pd.read_excel(xlsx, sheet_name=sheet_name, header=None)
    
    # getting the link of the webpage to include as the metadata 
    metadata = sheet_df.iat[0, 0]
    
    # Read the sheet into a DataFrame starting from the second row
    df = pd.read_excel(xlsx, sheet_name=sheet_name, skiprows=1)
    
    # Save the DataFrame to a CSV file
    csv_path = f'C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/{sheet_name}.csv'
    df.to_csv(csv_path, index=False)
    csv_files.append((csv_path, metadata))

# Display the list of generated CSV files and their metadata
csv_files

['University Advising Center FAQs', 'Student Financial Services FAQs', 'Parent FAQs', 'SMU Experience FAQs', 'UG Admissions Academics FAQs']


[('C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/University Advising Center FAQs.csv',
  'https://www.smu.edu/provost/saes/academic-support/university-advising-center/frequently-asked-questions'),
 ('C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/Student Financial Services FAQs.csv',
  'https://www.smu.edu/provost/saes/academic-support/student-academic-success/faq'),
 ('C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/Parent FAQs.csv',
  'https://www.smu.edu/provost/saes/academic-support/university-advising-center/incoming-students/for-parents'),
 ('C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/SMU Experience FAQs.csv',
  'https://www.smu.edu/admission/campuslife/faqlivingoncampus'),
 ('C:/Users/yawbt/OneDrive/Documents/GitHub/SURF-Project_Optimizing-PerunaBot/Data/UG Admissions Academics FAQs.csv',
  'https://www.smu.edu/admission/academics/faqsacademics'

In [16]:
# Now turning each csv into a langchain document
from langchain.document_loaders import CSVLoader

# Create LangChain documents from CSV files with metadata
csv_documents = []

for csv_path, metadata in csv_files:
    loader = CSVLoader(file_path=csv_path)
    csv_docs = loader.load()
    for csv_doc in csv_docs:
        csv_doc.metadata['source'] = metadata
    csv_documents.extend(csv_docs)

# Display the first document as an example
print(csv_documents[0])

page_content="question: What's the difference between all-college GPA and SMU GPA?\nanswer: Your all-college GPA is the GPA used from all your grades at any college or university you’ve attended, including SMU. \nYour SMU GPA is your GPA based only on your SMU grades. Some schools/majors use your all-college GPA or grades in courses you’ve\n taken at another institution to determine if they will admit you to their major." metadata={'source': 'https://www.smu.edu/provost/saes/academic-support/university-advising-center/frequently-asked-questions', 'row': 0}


Then lastly upload the csv documents into each vector store collection

In [17]:
# vector store collection 1 - uses parent/child text splitter with parent retriever
parent_retriever.add_documents(csv_documents)

In [18]:
# vector store collection 2 - uses semantic text splitter (or chunker) with the ensemble retriever (BM25 + vector store as retriever)
# uploaded to vector store using vector store as the retriever
vector_store_2_retriever.add_documents(csv_documents)

['0495c2dd9f2a44c1a995f57a6361afd1',
 'cd7aa5bef49c46479b2a8dfbfdae6e55',
 '651ba19a5321439a9c300da63b2eaa8a',
 '883ea6b136bf4f7388558b7ce0563d82',
 '304e9c0a056a44f8b29bd58bbf1ab433',
 '4f9ae18ce0004043b6c75777c51aabdb',
 '822a25ab44d64210aed6a4375150a7ca',
 '576ec6f17e954ff4b596480ea561052b',
 'bef6387675dc427b82aef9a8850a98b0',
 'e1b89c0dd40445e0b0f0e56414514882',
 'b5c8fcc0caf04dc39418959958cc3557',
 'a6d857ecf6ca462384e57446b2ae969c',
 '2d7f9f9107604979bc29b5601bcf1101',
 '09ccd958f83f466c8804846c3ec69ca8',
 '607e1a56f0184ceaaf888c86a09530a6',
 'd2458f8908004d7a82eb7c2adc3eb35a',
 'c2934d2a18424895998b63a7960b5d1a',
 'eeca8091f0924a2790fd2f554dd2138b',
 'cf74fb8c8e4f489c949cb49a49b05747',
 'aa1f978fbc1746a4bbe591a74be83eb7',
 '7758d70521d54609967be0e0e6028c44',
 'e2c60ecc409e4b798430704dff0bcf50',
 '7e885c6aa10a428ea8e5b843af4e9b0d',
 'ee5c833285e94df1bf52c687ca3de2de',
 '47ed9f9cf55747d5ac3a72f495212878',
 '7f97c10f55984339a7a7d00b86332fb5',
 'f65b356062374d669c23e7ca3d3fd184',
 

In [19]:
# vector stoer collection 0 - uses the recursive chatacter text splitter with vector store as the retriever
# base option from last project
vector_store_0_retriever.add_documents(csv_documents)

['e578696465aa43c79be665fb0f17d409',
 'ac0a95a22ff745cc8ad441ec84b0691c',
 'e4ff1be4b8fc4113a01494d51e6b74cb',
 '5b265797fde14bde9db11d1fb2f38473',
 '66acd6b8476849c7be6025cd8b1cfec5',
 '0fde38caa77f440a9b50d4333575e0b6',
 'b7fb807bc5ed4fe6b3a88d5e946f005f',
 'ac802659dab348968bf22dac9062eff9',
 '43069f562150432cb8f17d9a50f439a6',
 'aba64f42688b4256923d6e7d6a89fcb8',
 'a7f941e013ad4f5fb71f335be71e6dc6',
 '9bd8c169539e4b678e80f6f1d2cf92a8',
 '934f678a675f46edbd8ee95d5cd8a6a9',
 '1083a2c1a7e84b2984be7a9586026c71',
 '5a2d8124dced4986b739505e964e45f0',
 '53ff92facb9940dc8c1dc69b3137dc0f',
 '8c7eafa5db2d422aaa869b4423f7e3b6',
 '4f124034b3624f9db522f05210682bb7',
 'e9589611fbd846938fecd03204a77a87',
 '78d746e21c0742d18cfca6616766dc16',
 '38dab5c6faef4f5ba36daef625dcdb39',
 'd8c01fae34eb46d7b649f7a585badf2b',
 'c82e5cc8cb4b47dab8838debfce84748',
 'f8f34cca489b4373ae3f8715050ced64',
 '4b4824a663b04643920b092939818303',
 '5251d4b4dfa04f54bc6e364fc1036873',
 '7ec37de7f98342b993ea7dc3e81feda5',
 

Now we are officially done with the data-preprocessing step and can move on to the RAG pipelines for each model!